# 新聞分類模型

## 1. 爬取中央社新聞

In [2]:
import re
from bs4 import BeautifulSoup
import requests
import html5lib
import json

In [3]:
url = 'https://www.cna.com.tw/list/aspt.aspx'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html5lib')
n=0
labels=[]
while n < len(soup.select_one('#pnProductNavContents > ul').findAll('li')):
    try :
        lab = soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'}).text
        url =soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'})['href']
    except:(TypeError,IndentationError)
    labels.append((url,lab))
    n += 1
labels = list(set(labels))
labels.remove(('http://cnavideo.cna.com.tw', '影音'))
labels.remove(('/list/aall.aspx', '即時'))
labels.append(('https://www.cna.com.tw/list/aall.aspx', '即時'))
#labels.remove(('https://www.cna.com.tw/keyword/九合一.aspx', '九合一'))
labels.remove(('https://www.cna.com.tw/list/sp.aspx', '專題'))
labels.remove(('https://www.cna.com.tw/list/aall.aspx', '即時'))

labs = []
for item in labels:
    item = [item[0],item[1]]

    item[0] = re.search(r'list\/\w+',item[0]).group()[5:]
    labs.append(item)

### 使用payload()post中央社網站分類底下各標題的href連結，整理成labs_href
```python
labs_href=
[
    [ [href,href,href,.....],類別1 ],
    [ [href,href,href,.....],類別2 ],
    [ [href,href,href,.....],類別3 ],
    ...
]
```

In [4]:
def payload(item):
    payload={
        'id': '1056893641102242',
    'ev': 'Microdata',
    'dl': item[0],
    'rl': 'https://www.cna.com.tw/',
    'if': 'false',
    'ts': '1541087113735',
    'cd[Schema.org]': [],
    #'cd[OpenGraph]': {"og:url":"https://www.cna.com.tw/list/ahel.aspx","og:title":"生活 | 中央社 CNA","og:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","og:image:height":"200","og:image:width":"200","og:image":"https://img5.cna.com.tw/www/images/pic_fb.jpg","og:type":"article","article:author":"https://www.facebook.com/cnanewstaiwan","article:publisher":"https://www.facebook.com/cnanewstaiwan"}
    #'cd[Meta]': {"title":"\n\t生活 | 中央社 CNA\n","meta:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","meta:keywords":"News, 新聞, 即時新聞, 中央社"}
    'cd[DataLayer]': [],
    'cd[JSON-LD]': [],
    'sw': '1280',
    'sh': '5000',#'720',
    'v': '2.8.30',
    'r': 'stable',
    'a': 'tmgoogletagmanager',
    'ec': '1',
    'o': '30',
    'fbp': 'fb.2.1541074934894.620990608',
    'it': '1541087112886',
    'coo': 'false',
    'es': 'automatic',
    }
    return payload

In [5]:
labs_href=[]
for item in labs:
    soups=[]
    n=1
    while n < 13:
        url = 'https://www.cna.com.tw/cna2018api/api/simplelist/categorycode/'+item[0]+'/pageidx/'+str(n)
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        body = soup.findAll('body')[0].getText()
        PageUrl = re.findall(r'\"PageUrl\"\:\S[^\,]+',body)
        urls = [txt[11:-1] for txt in PageUrl]
        soups.extend(urls)
        n += 1
        if len(urls) < 20:
            break
    labs_href.append([soups,item[1]])

### 對labs_href中每一個href進行爬蟲抓出新聞標題與內文，並將標題、內文、類別整理成newslist
```python
newslist=
[
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    [ [標題,內文],類別 ]
    ...
]

```

In [158]:
news={}
newslist=[]
for lab in labs_href:
    lab_name = lab[1]
    lab_urls = lab[0]
    n = 0
    for url in lab_urls:
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        txt = ""
        try:
            title = soup.find_all('div',{'class':'centralContent'})[0].find_all('h1')[0].getText()
            for s in soup.find_all('div',{'class':'paragraph'})[0].findAll('p'):
                txt=txt+s.getText()
        except:IndexError
        research = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt)
        if research == None or len(research.group())<2:
            print(txt)
        else:
            txt = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt).group()[1:]

        page = [[title,txt],lab_name]
        newslist.append(page)
        news[(title,txt)]=lab_name
        n += 1
        if n % 20 == 0:
            print((lab_name,n),end=",")

('產經', 20),('產經', 40),('產經', 60),('產經', 80),('產經', 100),('兩岸', 20),('兩岸', 40),中央社駐香港特派員張謙/12月3日4年前「占中運動」爆發時，香港泛民主派一度氣勢如虹，並於2016年底的立法會選舉取得不錯成績，但近年來形勢逆轉，泛民面對不少內憂壓力。挾著年輕力量的崛起，泛民於2016年9月的立法會選舉大有斬獲，共取得26席。當中包括一人一票的分區直選議席，以及以業界區分的功能組別議席。功能組別以業界人士或代表區分，從來都是泛民較弱的版塊，但當年泛民在這方面獲得的選票大增，顯示其勢力正往業界延伸，威脅親政府建制派的基本盤。但「占中」之後，當局在北京中央高喊必須對「港獨」零容忍的情況下，開始以行政手段限制立場較為偏激的泛民人士參選。主要要求各候選人在參選前，必須簽署一份參選聲明，表明自己擁護基本法和保證效忠香港特別行政區。這項新規定導致一些泛民代表、特別是主張「港獨」的年輕代表，無法參加當年的立法會選舉，甚至讓新成立的年輕政黨香港眾志，一夜之間被排除在選舉之外，無法透過參選而進入香港管治架構。選後，當局也對泛民當選者窮追猛打，以他們在宣誓就任時未符合基本法所規定的「莊嚴」要求，向法院提出聲請，要求褫奪他們的資格，結果導致泛民6人喪失議席。進入2018年，泛民的困境並未消除。在立法會議員補選中未能拿回失去的全部議席，僅取回3席，失去另外2席，讓泛民在議會的人數減少。今年3月和11月，當局為泛民失去的其中5個議席進行補選，其中3月補選4人，11月補選1人。結果泛民在3月只贏回兩席，失去另外2席。更重要的是，泛民在一人一票的地方直選，被建制派打敗而失去1席，這是泛民和建制派歷來在一對一的對決下，首次落敗。在上月的補選中，泛民同樣在地方直選中落敗，受挫的的更是泛民元老級人馬、工黨成員李卓人。對於泛民連續兩次在一人一票地方直選、一對一對決之下敗給建制派，有學者解讀為泛民和建制的政治版圖正在發生變化。香港輿論和媒體一向認為，自香港主權轉移以來，泛民和建制無論在立法會或區議會的選舉，得票總和多是6比4，即前者得6成票，後者4成票。所以，每逢直選選區只有泛民和建制對決，建制派必定落敗，這種現象的背後，反映出兩派的民意基礎和支持度有差距。對於泛民連續兩次在直選落敗，香港中文大學政治與行政學系高級講師蔡子強說，這已非

## 2. 找出所有類別的高頻字

### 將newslist各種分類的標題與內文整合成title_tags , txt_tags
```python
title_tags=
{
    類別 : 全標題文字
}

txt_tags=
{
    類別 : 全內文文字
}
```

In [13]:
import jieba
import nltk

In [14]:
stopwords=[]
with open('../30/txt/stopwords_ch.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

In [15]:
lab_tags ={}
prelab = ''
books = list(news.items())
for book in books:
    for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
        if book[1]==lab:
            if prelab != lab:
                txt=''
            txt = txt+book[0]
            lab_tags[book[1]] = txt
            prelab = lab

In [16]:
filterwords=[
    '年','月','中央社','記者','編輯','譯者','日電',
    '\n',' ','「','」'
    ,'今天','說','表示','年','月','日','後','前','時','應','中','今年'
    ,'兩','下','人','最','次','再','先','盡量'
]

In [17]:
articles = []
titles = []
for news in newslist:
    articles.append((news[0][1],news[1]))
    titles.append((news[0][0],news[1]))

In [18]:
books = []

n = 0
while n < len(titles):
    head_dic = {}
    txt_dic = {}
    head = titles[n]
    txt = articles[n]
    
    head_dic[head[0]] = head[1]
    txt_dic[txt[0]] = txt[1]
    books.append([head_dic,txt_dic])
    n += 1

In [19]:
def get_tags(books=books):

    prelab = ''
    tags = {}
    for book in books:
        head_dic = book[0]
        txt_dic = book[1]
        n=0
        for dic in [head_dic,txt_dic]:

            for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
                if list(dic.values())[0]==lab:
                    if prelab != lab:
                        txt=''
                    txt += list(dic.keys())[0]
                    tags[list(dic.values())[0]] = txt
                    prelab = lab
            if n == 0:
                title_tags = tags
            elif n == 1:
                txt_tags = tags
            n += 1
    return title_tags,txt_tags

In [20]:
title_tags,txt_tags = get_tags(books)

In [165]:
title_tags

{'兩岸': '自媒體網路訛詐  公安部：三類成員四盈利模式中國公安部今年加大力度打擊自媒體「網路水軍」訛詐案，迄今已關閉大V帳號1100多個，並歸結出自媒體水軍的3類成員分工、4種訛詐盈利模式，以及4大犯罪運作特點。央視新聞引述公安部報導稱，為切實維護網路安全和民眾合法權益，今年以來，公安部成功偵破自媒體「網路水軍」團夥犯罪案件28起，抓獲嫌犯67名，關閉涉案網站31家，關閉各類網路大V帳號1100多個，涉及被敲詐勒索的企事業單位80多家。報導指出，公安機關調查發現，自媒體「網路水軍」人員構成相對複雜，概分3類。一是核心人員，主要包括網路公關公司及其雇用的「寫手」和「水軍」。網路公關公司是網路水軍的幕後老闆，負責接受客戶請求，策劃組織網路炒作、有償刪帖等活動。「寫手」熟悉網民心理，專職撰寫、提供炒作素材；「水軍」是網上炒作活動的具體實施者，以網上有償發帖牟利。第二是上游人員，即網路水軍業務的需求者，主要包括廣告商、委託人、爆料人。廣告商透過「水軍」炒作提高其投放廣告的點擊量；委託人、爆料人提供炒作素材，透過水軍攻擊炒作指定單位、人員，達到某些訴求。第三是下游人員，即網路水軍業務的「輔助實施者」，主要由專業推手、小型非法網站運營者和知名網站「內鬼」構成。專業推手多是網路「大V」、「網紅」等，借助自身在粉絲中的影響力，為炒作活動助威。小型非法網站運營者、知名網站的編輯或版主等「內鬼」，主要協助網路水軍刪除或置頂帖文，從中謀取非法利益。至於自媒體網路水軍的盈利模式，主要有4種：第一種是有償刪帖。先非法建立小型網站，針對地方企事業單位、個人，依託爆料人捏造事實、甚至惡意誹謗，並利用網路推手大肆炒作，脅迫涉事人員和企事業單位花錢消災。第二種是有償發帖。借助開辦的網路公司，承接客戶發帖業務，透過雇用的網路水軍，有目的有計劃地大規模炒作。第三種是非法廣告宣傳。即透過雇用網路水軍，並依託有關係的「網路大V」、知名博主、論壇版主、網紅等，為客戶轉發非法廣告、擴大宣傳效應。第四種是惡意傳播木馬病毒，提高點擊率。即透過將木馬病毒植入網頁，刷新網頁點擊率以博取廣告商目光，招攬業務、獲取經濟利益。此外，自媒體敲詐勒索作為傳統「網路水軍」變種，還具有4個特點：一是作案手段隱蔽，手法隱晦。涉案網站多搭建在境外，使用虛假身分辦理銀行卡、支付寶、4G網卡。作案手法專業，已形成集假網站製作、假身分銷

### 將title_tags，txt_tags中全文字斷詞為tokens，統計tokens找出高頻字整理為lab_fwords，lab_Ftwowords

In [21]:
def get_fwords0(tags={},top=300):


    lab_fwords={}
    lab_Ftwowords = {}
    for labname in tags.keys():
        print(labname,end=',')
        lab_words = tags[labname]
    #    seg_list = jieba.cut_for_search(lab_words)#, cut_all=False)
        seg_list = jieba.cut(lab_words, cut_all=False)
        txt = ",".join(seg_list)
        tokens = txt.split(',')
        tokens = list(filter(lambda a: a not in stopwords and a not in filterwords and a.isalpha() == True,tokens))#and len(a)>1, tokens))
        ###
        lab_alltwowords = list(nltk.bigrams(tokens))
        two = nltk.FreqDist(lab_alltwowords).most_common(top)
        lab_Ftwowords[labname] = [i[0]+i[1] for i,j in two]
        ###
        lab_fwords[labname] =  [i[0] for i in nltk.FreqDist(tokens).most_common(top)]
    return lab_fwords,lab_Ftwowords

In [203]:
def get_fwords(tags={},top=300):


    lab_fwords={}
#    lab_Ftwowords = {}
    for labname in tags.keys():
        print(labname,end=',')
        lab_words = tags[labname]
        seg_list = jieba.cut(lab_words, cut_all=False)
        txt = ",".join(seg_list)
        tokens = txt.split(',')
        tokens = list(filter(lambda a: a not in stopwords and a not in filterwords and a.isalpha() == True,tokens))#and len(a)>1, tokens))
        lab_alltwowords = list(nltk.bigrams(tokens))
        two_tokens =  [(i[0]+i[1]) for i in lab_alltwowords]
        tokens.extend(two_tokens)
        lab_fwords[labname] =  [i[0] for i in nltk.FreqDist(tokens).most_common(top)]
    return lab_fwords

In [204]:
txt_fwords3 =get_fwords(txt_tags)
txt_fwords3

產經,兩岸,政治,社會,文化,運動,娛樂,證券,國際,地方,科技,生活,

{'兩岸': ['中國',
  '報導',
  '香港',
  '大陸',
  '兩岸',
  '企業',
  '台灣',
  '已',
  '經濟',
  '美國',
  '政府',
  '國家',
  '指出',
  '名',
  '北京',
  '發展',
  '元',
  '新',
  '中共',
  '相關',
  '進行',
  '工作',
  '合作',
  '包括',
  '問題',
  '新疆',
  '要求',
  '代表',
  '中國大陸',
  '社會',
  '改革',
  '人員',
  '論壇',
  '種',
  '人士',
  '關係',
  '發生',
  '政策',
  '政治',
  '當局',
  '網路',
  '公司',
  '認為',
  '疫情',
  '憲法',
  '數據',
  '遭',
  '委員會',
  '泛民',
  '就業',
  '中美',
  '支持',
  '曾',
  '新聞',
  '人民幣',
  '占',
  '媒體',
  '目前',
  '習近平',
  '官方',
  '情況',
  '交流',
  '國際',
  '提出',
  '上海',
  '活動',
  '措施',
  '大學',
  '項',
  '鐵路',
  '調查',
  '主要',
  '組織',
  '官員',
  '強調',
  '方面',
  '透過',
  '影響',
  '電台',
  '自由',
  '指',
  '市場',
  '舉行',
  '首',
  '選舉',
  '些',
  '顯示',
  '部門',
  '中心',
  '主任',
  '全國',
  '陸委會',
  '引述',
  '民眾',
  '公布',
  '進口',
  '警方',
  '民主',
  '發表',
  '事件',
  '台',
  '非洲',
  '產業',
  '中央',
  '出現',
  '議員',
  '雙方',
  '城市',
  '部',
  '指數',
  '豬瘟',
  '非洲豬瘟',
  '以來',
  '資訊',
  '憲章',
  '部分',
  '教育',
  '億',
  '文化',
  '希望',
  '學生',
  '管理',
  '研究',
  '持續',
  '實施',
 

In [211]:
#去除交集的高頻字
def remove_mix(t):
    lab_fwords = t
#    lab_Ftwowords = t[1]


    dic3={}
    for lab in lab_fwords.keys():   
        word_content2 = lab_fwords[lab].copy()
        for lab2 in lab_fwords.keys():        
            if lab != lab2:
                word_content = [w for w in lab_fwords[lab] if (w in [a for a in lab_fwords[lab2]])]
                for w in word_content:
                    try:
                        word_content2.remove(w)                  
                    except ValueError:
                        pass
        dic3[lab] = word_content2  



#    dic2={}
#    for lab in lab_Ftwowords.keys():   
#        word_content2 = lab_Ftwowords[lab].copy()
#        for lab2 in lab_Ftwowords.keys():        
#            if lab != lab2:
#                word_content = [w for w in lab_Ftwowords[lab] if (w in [a for a in lab_Ftwowords[lab2]])]
#                for w in word_content:
#                    try:
#                        word_content2.remove(w)                   
#                    except ValueError:
#                        pass
#        dic2[lab] = word_content2
        
    return dic3

In [214]:
title_fwords = remove_mix(get_fwords(title_tags,700))
txt_fwords = remove_mix(get_fwords(txt_tags,800))

產經,兩岸,政治,社會,文化,運動,娛樂,證券,國際,地方,科技,生活,產經,兩岸,政治,社會,文化,運動,娛樂,證券,國際,地方,科技,生活,

In [25]:
def get_allFtwowords(lab_Ftwowords={}):
    
    
    All_Ftwowords2=[]
    for n in lab_Ftwowords.values():
        for m in n:
            All_Ftwowords2.append(m)

    All_Ftwowords = []
    for w in All_Ftwowords2:
        All_Ftwowords.append(w)
    return All_Ftwowords

In [26]:
#title_allFtwo = get_allFtwowords(title_Ftwowords)
#txt_allFtwo = get_allFtwowords(txt_Ftwowords)

In [27]:
import random
random.shuffle(books)

In [218]:
def pos_title_features(article,top):
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]

    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))
  
 #    twowords = list(nltk.bigrams(article_tokens))
#    alltwowords2 = [i[0]+i[1] for i in twowords]
#    intwowords = [two for two in alltwowords2 if two in title_allFtwo]
#    ftwowords = nltk.FreqDist(intwowords).most_common(top)
    toks = [w[0] for w in nltk.FreqDist(filt_tokens).most_common(top)]

#    if len([i[0] for i in ftwowords if i[0] in [w for li in title_Ftwowords.values() for w in li]])==0 and len([i for i in toks if i in [w for li in title_fwords.values() for w in li]])==0:
#        for i in ftwowords:
#            w = i[0]
#            features[w] = True
            
#    else:
#        for i in ftwowords:
#            w = i[0]
#            if w in [w for li in title_Ftwowords.values() for w in li]:
#                features[w] = True

    if len([i for i in toks if i in [w for li in title_fwords.values() for w in li]])==0 :#and len([i[0] for i in ftwowords if i[0] in [w for li in title_Ftwowords.values() for w in li]])==0:
        for i in toks[:int(top)]:
            features[i] = True
    else:
        bigcontent=[]
        kk=''
        for k in title_fwords.keys():
            content=[]
            for tok in toks:
                if tok in title_fwords[k] :
                    content.append(tok)
                if len(bigcontent) < len(content):
                    bigcontent = content.copy()
                    kk = k

        for tok in bigcontent:  
            features[tok] = True
    return features

In [219]:
pos_title_features(test_title,200)

{'交易': True,
 '名家': True,
 '朝核': True,
 '用孟晚': True,
 '舟': True,
 '觀點': True,
 '貿易': True,
 '貿易戰': True}

In [133]:
def get_title_featuresets(book,top=200):
    n = 0
    head_dic = book[0]
    article = list(head_dic.keys())[0]
    labname = list(head_dic.values())[0]
    features = pos_title_features(article,top)

    if len(list(features.keys())) < 3:
        feature_item = [{},labname]
    else:
        feature_item = [features,labname]

    return feature_item

In [134]:
def pos_txt_features(article,top):
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]

    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))#and len(a)>1, article_tokens))

    twowords = list(nltk.bigrams(article_tokens))
    alltwowords2 = [i[0]+i[1] for i in twowords]
    intwowords = [two for two in alltwowords2 if two in txt_allFtwo]
    ftwowords = nltk.FreqDist(intwowords).most_common(int(top))
    toks = [w[0] for w in nltk.FreqDist(filt_tokens).most_common(top)]


    
    if len([i for i in toks if i in [w for li in txt_fwords.values() for w in li]])==0 and len([i[0] for i in ftwowords if i[0] in [w for li in txt_Ftwowords.values() for w in li]])==0:
        for i in toks[:int(len(toks)*0.25)]:
            features[i] = True
    else:
        bigcontent=[]
        kk=''
        for k in txt_fwords.keys():
            content=[]
            for tok in toks:
                if tok in txt_fwords[k] :
                    content.append(tok)
                if len(bigcontent) < len(content):
                    bigcontent = content.copy()
                    kk = k
        for tok in bigcontent:  
            features[tok] = True

            
    if len([i[0] for i in ftwowords if i[0] in [w for li in txt_Ftwowords.values() for w in li]])==0 and len([i for i in toks if i in [w for li in txt_fwords.values() for w in li]])==0:
        for i in ftwowords[:int(len(ftwowords)*0.25)]:
            w = i[0]
            features[w] = True           
    else:
        bigcontent=[]
        kk2=''
        for k in txt_Ftwowords.keys():
            content=[]
            for i in ftwowords:
                tok = i[0]
                if tok in txt_Ftwowords[kk] :
                    content.append(tok)
                if len(bigcontent) < len(content):
                    bigcontent = content.copy()
            for tok in bigcontent:  
                features[tok] = True

    return features

In [135]:
from collections import Counter
def get_top(article,top,get=5):
    features={}
    t = ''
    seg_list = jieba.cut_for_search(article)
    t = ",".join(seg_list)
    article_tokens = t.split(',')
    article_tokens = [w for w in article_tokens if w.isalpha()]

    filt_tokens = list(filter(lambda a: a not in stopwords and a not in filterwords , article_tokens))
    twowords = list(nltk.bigrams(article_tokens))
    alltwowords2 = [i[0]+i[1] for i in twowords]
    intwowords = [two for two in alltwowords2 if two in txt_allFtwo]

    ftwowords = nltk.FreqDist(intwowords).most_common(int(top))
    toks = [w for w in nltk.FreqDist(filt_tokens).most_common(top)]

    selectwords = ([t for t in toks if t[0] in [w for li in txt_fwords.values() for w in li]])
    klist=[]
    for w in selectwords[:get]:  
        for k in txt_fwords.keys():
            if w[0] in [s for s in txt_fwords[k]] :
                klist.append(k)
                features['top'] = Counter(klist).most_common(1)[0][0]

    return features

In [136]:
def get_txt_featuresets(book,top=200):
    n = 0
    txt_dic = book[1]
    article = list(txt_dic.keys())[0]
    labname = list(txt_dic.values())[0]
    features = pos_txt_features(article,top)
    feature_item = [features,labname]

    return feature_item

In [137]:
def get_featuresets(books=books,top=200):
    title_featuresets = []
    txt_featuresets = []
    for book in books:

        title_featureset = get_title_featuresets(book,top)
        title_featuresets.append(title_featureset)
        
        txt_featureset = get_txt_featuresets(book,top)
        txt_featuresets.append(txt_featureset)
    return title_featuresets, txt_featuresets

In [138]:
title_featuresets, txt_featuresets = get_featuresets(top=170)

In [139]:
title_featuresets = title_featuresets.copy()
size = int(len(title_featuresets)*0.7)
train_set, test_set = title_featuresets[:size], title_featuresets[size:]
title_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(title_classifier, test_set)

0.40555555555555556

In [140]:
txt_featuresets = txt_featuresets.copy()
size = int(len(txt_featuresets)*0.7)
train_set, test_set = txt_featuresets[:size], txt_featuresets[size:]
txt_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(txt_classifier, test_set)

0.8527777777777777

In [141]:
title_comanswer = title_classifier.classify_many([i[0] for i in title_featuresets])
txt_comanswer = txt_classifier.classify_many([i[0] for i in txt_featuresets])

title_answer = [i[1] for i in title_featuresets]
txt_answer = [i[1] for i in txt_featuresets]

txt_top = [list(get_top(txt,200).values())[0] for txt in [list(book[1].keys())[0] for book in books]]

In [142]:
import pandas as pd
import numpy as np

In [143]:
title_correct = ['F' if w != title_answer[title_comanswer.index(w)] else 'T' for w in title_comanswer ]
txt_correct = ['F' if w != txt_answer[txt_comanswer.index(w)] else 'T' for w in txt_comanswer ]
df = pd.DataFrame([txt_comanswer,txt_answer,txt_top,txt_correct,title_comanswer,title_answer,title_correct],index=['txt_comanswer','txt_answer','txt_top','txt_correct','title_comanswer','title_answer','title_correct']).T

In [144]:
df.head(10)

,txt_comanswer,txt_answer,txt_top,txt_correct,title_comanswer,title_answer,title_correct
0,證券,產經,產經,F,國際,產經,F
1,證券,證券,證券,F,國際,證券,F
2,政治,政治,政治,T,政治,政治,T
3,證券,產經,產經,F,產經,產經,T
4,生活,生活,生活,T,國際,生活,F
5,社會,社會,社會,T,社會,社會,T
6,科技,科技,科技,T,科技,科技,T
7,運動,運動,運動,T,運動,運動,T
8,娛樂,娛樂,娛樂,T,娛樂,娛樂,T
9,科技,產經,科技,T,國際,產經,F


In [145]:
from nltk.classify import SklearnClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC

In [146]:
dataset = []

answer = ""
for i,v in df.iterrows():
    answer_dict={}
    print(v)
    answer_dict['txt_comanswer']=v[0]
    answer_dict['txt_top']=v[2]
    answer_dict['title_comanswer']=v[4]

    answer = v[1]
    dataset.append((answer_dict,answer))

txt_comanswer      證券
txt_answer         產經
txt_top            產經
txt_correct         F
title_comanswer    國際
title_answer       產經
title_correct       F
Name: 0, dtype: object
txt_comanswer      證券
txt_answer         證券
txt_top            證券
txt_correct         F
title_comanswer    國際
title_answer       證券
title_correct       F
Name: 1, dtype: object
txt_comanswer      政治
txt_answer         政治
txt_top            政治
txt_correct         T
title_comanswer    政治
title_answer       政治
title_correct       T
Name: 2, dtype: object
txt_comanswer      證券
txt_answer         產經
txt_top            產經
txt_correct         F
title_comanswer    產經
title_answer       產經
title_correct       T
Name: 3, dtype: object
txt_comanswer      生活
txt_answer         生活
txt_top            生活
txt_correct         T
title_comanswer    國際
title_answer       生活
title_correct       F
Name: 4, dtype: object
txt_comanswer      社會
txt_answer         社會
txt_top            社會
txt_correct         T
title_comanswer    社會
title

Name: 114, dtype: object
txt_comanswer      證券
txt_answer         證券
txt_top            證券
txt_correct         F
title_comanswer    證券
title_answer       證券
title_correct       T
Name: 115, dtype: object
txt_comanswer      政治
txt_answer         政治
txt_top            政治
txt_correct         T
title_comanswer    政治
title_answer       政治
title_correct       T
Name: 116, dtype: object
txt_comanswer      產經
txt_answer         產經
txt_top            產經
txt_correct         T
title_comanswer    產經
title_answer       產經
title_correct       T
Name: 117, dtype: object
txt_comanswer      娛樂
txt_answer         娛樂
txt_top            娛樂
txt_correct         T
title_comanswer    娛樂
title_answer       娛樂
title_correct       T
Name: 118, dtype: object
txt_comanswer      國際
txt_answer         產經
txt_top            國際
txt_correct         T
title_comanswer    國際
title_answer       產經
title_correct       F
Name: 119, dtype: object
txt_comanswer      地方
txt_answer         地方
txt_top            地方
txt_correct   

txt_comanswer      科技
txt_answer         科技
txt_top            科技
txt_correct         T
title_comanswer    科技
title_answer       科技
title_correct       T
Name: 227, dtype: object
txt_comanswer      運動
txt_answer         運動
txt_top            運動
txt_correct         T
title_comanswer    運動
title_answer       運動
title_correct       T
Name: 228, dtype: object
txt_comanswer      科技
txt_answer         科技
txt_top            科技
txt_correct         T
title_comanswer    科技
title_answer       科技
title_correct       T
Name: 229, dtype: object
txt_comanswer      科技
txt_answer         科技
txt_top            科技
txt_correct         T
title_comanswer    國際
title_answer       科技
title_correct       F
Name: 230, dtype: object
txt_comanswer      地方
txt_answer         地方
txt_top            地方
txt_correct         T
title_comanswer    地方
title_answer       地方
title_correct       T
Name: 231, dtype: object
txt_comanswer      兩岸
txt_answer         兩岸
txt_top            兩岸
txt_correct         T
title_comanswer  

Name: 348, dtype: object
txt_comanswer      國際
txt_answer         國際
txt_top            國際
txt_correct         T
title_comanswer    國際
title_answer       國際
title_correct       F
Name: 349, dtype: object
txt_comanswer      生活
txt_answer         生活
txt_top            生活
txt_correct         T
title_comanswer    國際
title_answer       生活
title_correct       F
Name: 350, dtype: object
txt_comanswer      運動
txt_answer         運動
txt_top            運動
txt_correct         T
title_comanswer    運動
title_answer       運動
title_correct       T
Name: 351, dtype: object
txt_comanswer      生活
txt_answer         生活
txt_top            生活
txt_correct         T
title_comanswer    生活
title_answer       生活
title_correct       T
Name: 352, dtype: object
txt_comanswer      文化
txt_answer         文化
txt_top            文化
txt_correct         T
title_comanswer    文化
title_answer       文化
title_correct       T
Name: 353, dtype: object
txt_comanswer      國際
txt_answer         國際
txt_top            國際
txt_correct   

Name: 455, dtype: object
txt_comanswer      政治
txt_answer         政治
txt_top            政治
txt_correct         T
title_comanswer    國際
title_answer       政治
title_correct       F
Name: 456, dtype: object
txt_comanswer      科技
txt_answer         國際
txt_top            國際
txt_correct         T
title_comanswer    國際
title_answer       國際
title_correct       F
Name: 457, dtype: object
txt_comanswer      社會
txt_answer         社會
txt_top            社會
txt_correct         T
title_comanswer    國際
title_answer       社會
title_correct       F
Name: 458, dtype: object
txt_comanswer      國際
txt_answer         國際
txt_top            國際
txt_correct         T
title_comanswer    國際
title_answer       國際
title_correct       F
Name: 459, dtype: object
txt_comanswer      科技
txt_answer         科技
txt_top            科技
txt_correct         T
title_comanswer    國際
title_answer       科技
title_correct       F
Name: 460, dtype: object
txt_comanswer      證券
txt_answer         產經
txt_top            產經
txt_correct   

...

Name: 663, dtype: object
txt_comanswer      科技
txt_answer         科技
txt_top            科技
txt_correct         T
title_comanswer    科技
title_answer       科技
title_correct       T
Name: 664, dtype: object
txt_comanswer      生活
txt_answer         生活
txt_top            生活
txt_correct         T
title_comanswer    生活
title_answer       生活
title_correct       T
Name: 665, dtype: object
txt_comanswer      地方
txt_answer         地方
txt_top            地方
txt_correct         T
title_comanswer    地方
title_answer       地方
title_correct       T
Name: 666, dtype: object
txt_comanswer      政治
txt_answer         政治
txt_top            政治
txt_correct         T
title_comanswer    國際
title_answer       政治
title_correct       F
Name: 667, dtype: object
txt_comanswer      運動
txt_answer         運動
txt_top            運動
txt_correct         T
title_comanswer    國際
title_answer       運動
title_correct       F
Name: 668, dtype: object
txt_comanswer      文化
txt_answer         文化
txt_top            文化
txt_correct   

Name: 774, dtype: object
txt_comanswer      社會
txt_answer         國際
txt_top            國際
txt_correct         T
title_comanswer    國際
title_answer       國際
title_correct       F
Name: 775, dtype: object
txt_comanswer      證券
txt_answer         證券
txt_top            證券
txt_correct         F
title_comanswer    證券
title_answer       證券
title_correct       T
Name: 776, dtype: object
txt_comanswer      政治
txt_answer         生活
txt_top            生活
txt_correct         T
title_comanswer    生活
title_answer       生活
title_correct       T
Name: 777, dtype: object
txt_comanswer      運動
txt_answer         運動
txt_top            運動
txt_correct         T
title_comanswer    國際
title_answer       運動
title_correct       F
Name: 778, dtype: object
txt_comanswer      證券
txt_answer         證券
txt_top            證券
txt_correct         F
title_comanswer    證券
title_answer       證券
title_correct       T
Name: 779, dtype: object
txt_comanswer      兩岸
txt_answer         兩岸
txt_top            兩岸
txt_correct   

Name: 880, dtype: object
txt_comanswer      社會
txt_answer         社會
txt_top            社會
txt_correct         T
title_comanswer    社會
title_answer       社會
title_correct       T
Name: 881, dtype: object
txt_comanswer      國際
txt_answer         國際
txt_top            國際
txt_correct         T
title_comanswer    國際
title_answer       國際
title_correct       F
Name: 882, dtype: object
txt_comanswer      證券
txt_answer         證券
txt_top            證券
txt_correct         F
title_comanswer    證券
title_answer       證券
title_correct       T
Name: 883, dtype: object
txt_comanswer      國際
txt_answer         國際
txt_top            國際
txt_correct         T
title_comanswer    國際
title_answer       國際
title_correct       F
Name: 884, dtype: object
txt_comanswer      娛樂
txt_answer         娛樂
txt_top            娛樂
txt_correct         T
title_comanswer    國際
title_answer       娛樂
title_correct       F
Name: 885, dtype: object
txt_comanswer      產經
txt_answer         產經
txt_top            產經
txt_correct   

Name: 998, dtype: object
txt_comanswer      文化
txt_answer         文化
txt_top            文化
txt_correct         T
title_comanswer    國際
title_answer       文化
title_correct       F
Name: 999, dtype: object
txt_comanswer      娛樂
txt_answer         娛樂
txt_top            娛樂
txt_correct         T
title_comanswer    娛樂
title_answer       娛樂
title_correct       T
Name: 1000, dtype: object
txt_comanswer      政治
txt_answer         政治
txt_top            政治
txt_correct         T
title_comanswer    國際
title_answer       政治
title_correct       F
Name: 1001, dtype: object
txt_comanswer      證券
txt_answer         證券
txt_top            證券
txt_correct         F
title_comanswer    國際
title_answer       證券
title_correct       F
Name: 1002, dtype: object
txt_comanswer      文化
txt_answer         文化
txt_top            文化
txt_correct         T
title_comanswer    國際
title_answer       文化
title_correct       F
Name: 1003, dtype: object
txt_comanswer      政治
txt_answer         政治
txt_top            政治
txt_correc

txt_comanswer      兩岸
txt_answer         兩岸
txt_top            兩岸
txt_correct         T
title_comanswer    國際
title_answer       兩岸
title_correct       F
Name: 1111, dtype: object
txt_comanswer      娛樂
txt_answer         娛樂
txt_top            娛樂
txt_correct         T
title_comanswer    國際
title_answer       娛樂
title_correct       F
Name: 1112, dtype: object
txt_comanswer      兩岸
txt_answer         兩岸
txt_top            兩岸
txt_correct         T
title_comanswer    國際
title_answer       兩岸
title_correct       F
Name: 1113, dtype: object
txt_comanswer      兩岸
txt_answer         兩岸
txt_top            兩岸
txt_correct         T
title_comanswer    國際
title_answer       兩岸
title_correct       F
Name: 1114, dtype: object
txt_comanswer      兩岸
txt_answer         兩岸
txt_top            兩岸
txt_correct         T
title_comanswer    國際
title_answer       兩岸
title_correct       F
Name: 1115, dtype: object
txt_comanswer      國際
txt_answer         國際
txt_top            國際
txt_correct         T
title_comans

In [147]:
dataset

[({'title_comanswer': '國際', 'txt_comanswer': '證券', 'txt_top': '產經'}, '產經'),
 ({'title_comanswer': '國際', 'txt_comanswer': '證券', 'txt_top': '證券'}, '證券'),
 ({'title_comanswer': '政治', 'txt_comanswer': '政治', 'txt_top': '政治'}, '政治'),
 ({'title_comanswer': '產經', 'txt_comanswer': '證券', 'txt_top': '產經'}, '產經'),
 ({'title_comanswer': '國際', 'txt_comanswer': '生活', 'txt_top': '生活'}, '生活'),
 ({'title_comanswer': '社會', 'txt_comanswer': '社會', 'txt_top': '社會'}, '社會'),
 ({'title_comanswer': '科技', 'txt_comanswer': '科技', 'txt_top': '科技'}, '科技'),
 ({'title_comanswer': '運動', 'txt_comanswer': '運動', 'txt_top': '運動'}, '運動'),
 ({'title_comanswer': '娛樂', 'txt_comanswer': '娛樂', 'txt_top': '娛樂'}, '娛樂'),
 ({'title_comanswer': '國際', 'txt_comanswer': '科技', 'txt_top': '科技'}, '產經'),
 ({'title_comanswer': '國際', 'txt_comanswer': '政治', 'txt_top': '政治'}, '政治'),
 ({'title_comanswer': '地方', 'txt_comanswer': '地方', 'txt_top': '地方'}, '地方'),
 ({'title_comanswer': '社會', 'txt_comanswer': '社會', 'txt_top': '社會'}, '社會'),
 ({'title_co

In [148]:
size = int(len(dataset)*0.7)
train_data, test_data = dataset[:size], dataset[size:]
test_data2 = [i[0] for i in test_data]

classif0 = SklearnClassifier(BernoulliNB()).train(train_data)
classif = SklearnClassifier(SVC(), sparse=False).train(train_data)

In [149]:
ch = []
n=0
for f in dataset[size:] :
    ch.append([f,classif.classify_many(test_data2)[n]])
    n += 1
ch

[[({'title_comanswer': '國際', 'txt_comanswer': '證券', 'txt_top': '產經'}, '產經'),
  '產經'],
 [({'title_comanswer': '國際', 'txt_comanswer': '國際', 'txt_top': '國際'}, '國際'),
  '國際'],
 [({'title_comanswer': '國際', 'txt_comanswer': '娛樂', 'txt_top': '娛樂'}, '娛樂'),
  '娛樂'],
 [({'title_comanswer': '國際', 'txt_comanswer': '產經', 'txt_top': '地方'}, '地方'),
  '地方'],
 [({'title_comanswer': '國際', 'txt_comanswer': '娛樂', 'txt_top': '社會'}, '娛樂'),
  '娛樂'],
 [({'title_comanswer': '證券', 'txt_comanswer': '證券', 'txt_top': '證券'}, '證券'),
  '證券'],
 [({'title_comanswer': '娛樂', 'txt_comanswer': '文化', 'txt_top': '文化'}, '文化'),
  '文化'],
 [({'title_comanswer': '國際', 'txt_comanswer': '娛樂', 'txt_top': '娛樂'}, '娛樂'),
  '娛樂'],
 [({'title_comanswer': '社會', 'txt_comanswer': '社會', 'txt_top': '社會'}, '社會'),
  '社會'],
 [({'title_comanswer': '生活', 'txt_comanswer': '科技', 'txt_top': '生活'}, '兩岸'),
  '生活'],
 [({'title_comanswer': '娛樂', 'txt_comanswer': '娛樂', 'txt_top': '娛樂'}, '娛樂'),
  '娛樂'],
 [({'title_comanswer': '國際', 'txt_comanswer': '運動', 't

In [150]:
back = classif.classify_many(test_data2)
n = 0
p=0
while n < len(back):
    if back[n] ==  [i[1] for i in test_data][n]:
        p += 1
    n += 1
print(p)
print(p/len(back))

338
0.9388888888888889


In [151]:
def get_lab(title='',txt='',top=200):
    f = pos_txt_features(txt,top)
    t =  pos_title_features(title,top)
    top = list(get_top(txt,top).values())[0]

    if len(t.keys()) < 2:
        final = classif.classify_many({'txt_comanswer': txt_classifier.classify(f),'txt_top':top})
        final_title = ""
    elif title_classifier.classify(t) == txt_classifier.classify(f) and txt_classifier.classify(f)==top and top == title_classifier.classify(t):
        final = txt_classifier.classify(f)
        final_title = title_classifier.classify(t)
    else:
        final = classif.classify_many({'title_comanswer': title_classifier.classify(t), 'txt_comanswer': txt_classifier.classify(f),'txt_top':top})
        final_title = title_classifier.classify(t)
    print(f)
    print([final_title,txt_classifier.classify(f),top])

    return final

In [155]:
test_title="""
《名家觀點》用孟晚舟交易朝核或貿易戰？


"""

In [156]:
test_txt="""
華為CFO孟晚舟在加拿大被捕，嚴重性不僅影響中美貿易談判，且極可能衝擊中美關係，使本已脆弱的兩國關係雪上加霜。

孟晚舟的特殊性不但在她是華為創始人任正非之女，更在於華為是中國最成功企業，是中國高科技代表。在中國輿論場中，華為和任正非已被塑造成民族企業的象徵和模範生。美國此舉明顯針對華為，從中國角度看，華為是否違背美國有關伊朗禁令，不是事情關鍵，因為中國並不承認美國長臂管理的合法性。中方還認為，既然美國已經盯上華為，即使沒有這條理由，也會找出其他理由。

今年4月，當美國以中興違反禁令而對其痛下殺手時，導致中國民族主義群情激憤。但因中興在芯片等高科技的研發方面無所作為，網民認為它是咎由自取，可謂恨鐵不成鋼。儘管最後因川普的干預放了中興一條生路，然而代價不僅有高額罰金，美國還派合規團隊監督審查中興生意，被一些民族主義者視為「奇恥大辱」，間接損害了習近平權威。

而川普願放中興一馬，主要是從地緣政治角度考量，他還需要習近平幫忙，來擺平金正恩。相比中興，華為分量和地位無疑重要得多。美國指使加國逮捕華為高管，使事件比中興事件嚴峻得多，對中美都將是燙手山芋，處理不好會引發海嘯效應。

目前不清楚孟被捕川普是否事先知情。據悉，逮捕令是由紐約州檢察官辦公室發出，那麼此事有兩種可能，一是紐約州檢察官辦公室簽發逮捕令時沒有告知川普；二是告知了川普，後者沒有阻止或阻止了沒有起作用。

鑑於美國三權分立，檢察官要逮捕某個人時，無須通知行政部門。然而，鑑於孟的特殊性，要說檢察官辦公室完全不知道其決定很可能在中美引發災難性政治後果，似乎說不過去。合乎情理的假設是，檢察官把這事告知美國政府，但川普沒有阻止，若如此，川普就是有意利用這張牌來牽制中國。至於是用在貿易戰上，還是用在其他方面，則不知曉。

據報導，孟晚舟12月1日由加拿大警方逮捕。當天習近平正同川普共進晚餐。有理由認為習在那一時刻還未得知孟被捕，否則，習很難在貿易問題上向川普承諾作大讓步，除非當時他未認知此事將會對他造成何種困境。事情已過去五天，這五天裡有一種可能，即中方同美加交涉，試圖讓加方放人，如果人被放了，就權當這個事沒有發生，即使以後曝出，時過境遷，最多也就是一個談資而已。

但事已至此，孟晚舟沒有「犯事」的概率應該很少，否則美方沒法向中方交代。如果前述分析不錯，川普要用孟來「做局」，華盛頓有兩種處理方式，一是孟被判刑，同時對華為發出禁令；二是迫使北京作出讓步，換取孟被釋放，同時不對華為採取禁令。假如是第二種情況，讓步就不是小讓步，而一定是大讓步。問題是，習近平願做大讓步嗎？能做大讓步嗎？

看貿易戰方面，如果川普要以此要挾習近平，應是逼迫習全盤接受美國談判要求。此外，川普也可能要用它在北韓（朝鮮）問題上和習近平做一番交易。因為川普清楚，沒有習幫忙，他不能壓服金正恩棄核。而以中國對北韓命脈的掌控，習能做到這點。問題就在習近平願不願施壓金正恩。我認為，貿易談判中，習近平完全接受美國要求的可能性非常非常小，更大可能是在朝核問題上配合川普，因為北韓棄核從長遠看，也符合中國利益。

假如川普就是想在貿易上做交易，而習近平又不完全答應，最後結果極可能是雙方談崩。對習來說，無論貿易談判或朝核問題，如果他要拿華為做交易，都會引起國內民族主義和民粹主義強烈反彈，軍方也有可能對習心生不滿。習近平現在民意基礎是這兩股主義，他若不想失去他們的支持，就只能跟著強硬。一旦貿易談崩，美國如期加徵關稅，中美關係將全面倒退，世界秩序將進入一個動盪時期。

這對美國來說，當然亦非是最有利選擇。當下看，儘管美國已形成對中國的負面認知，但和中國打一場全面的冷戰，可能只是部分鷹派的想法，多數美國民眾理應還是想和中國鬥而不破。總之，不管這件事川普是否知情或干預，若處理不好，對美中都是傷害，未來的歷史，它可能會是一個標誌性事件。（作者是中國戰略分析智庫研究員）
"""

In [157]:
get_lab(title=test_title,txt=test_txt,top=200)

{'讓步': True, '禁令': True, '金正恩': True, '北韓': True, '任正': True, '此事': True, '此舉': True, '加國': True, '民族主義': True, '逮捕令': True, '報導孟晚': True, '種情況': True, '美國要求': True}
['產經', '國際', '國際']


['國際']